In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd

pd.set_option('display.max_columns', None)

### Use 73% personal travel fraction of light duty VMT (excluding light commercial trucks) to downscale Evolved VMT data. Then apply LATCH weights for spatial variation.

In [2]:
#Personal travel fraction is estimated by taking the national sum of household travel in 2017 from NHTS and dividing it by the national sum of light-duty vehicle travel from FHWA HPMS for 2017.

state = 'new mexico'
state_abbrev = 'NM'
personal_travel_fraction = .73

In [3]:
latch = pd.read_csv('../Transportation/latch_2017_updated.csv')
latch['geocode'] = latch['geocode'].astype(str)

In [4]:
co_inds = latch['geocode'].str.startswith('8')
nv_inds = latch['geocode'].str.startswith('32')
nm_inds = latch['geocode'].str.startswith('35')

latch_western = latch[co_inds | nv_inds | nm_inds]
latch_western['state'] = 0
latch_western.loc[co_inds,'state']='CO'
latch_western.loc[nv_inds,'state']='NV'
latch_western.loc[nm_inds,'state']='NM'

<ipython-input-4-e8fd32a5b1ef>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  latch_western['state'] = 0
/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [5]:
latch_western.rename(columns= {'geocode': 'census tract', 'est_vmiles': 'avg_weekday_hh_vmt', 'est_pmiles': 'avg_weekday_person_miles',
                              'est_ptrp': 'avg_weekday_person_trips', 'est_vtrp': 'avg_weekday_vehicle_trips'}, inplace=True)

latch_western = latch_western[['state','census tract','hh_cnt','median_hh_inc','tot_pop','avg_weekday_hh_vmt','avg_weekday_person_miles',
                               'avg_weekday_person_trips', 'avg_weekday_vehicle_trips','pct_worker_1', 'pct_worker_2_more','hh_veh_0',
                               'hh_veh_1','hh_veh_2_more',]]

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4295: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [6]:
#Uses household count from ACS 2012-2016 data (in LATCH dataset) to estimate tract-level weekday VMT.
latch_western['hh_w_veh']= latch_western['hh_cnt']-latch_western['hh_veh_0']
latch_western['pct_hh_w_veh'] = latch_western['hh_w_veh'] / latch_western['hh_cnt']
latch_western['tract_weekday_vmt'] = latch_western['avg_weekday_hh_vmt'] * latch_western['hh_w_veh']

In [7]:
#Drops census tracts with missing data.
missing_data_inds = latch_western['hh_cnt']==0
latch_western = latch_western[~missing_data_inds]

In [8]:
#Assumes distribution of weekday travel VMT can represent distribution of all light-duty auto VMT (weekday and weekend). 
state_fuel_burden = latch_western[latch_western['state']==state_abbrev]
state_fuel_burden['tract_vmt_weight']= state_fuel_burden['tract_weekday_vmt'] / state_fuel_burden['tract_weekday_vmt'].sum()

<ipython-input-8-59b6ba037cf4>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  state_fuel_burden['tract_vmt_weight']= state_fuel_burden['tract_weekday_vmt'] / state_fuel_burden['tract_weekday_vmt'].sum()


In [9]:
state_fuel_burden = state_fuel_burden[['tract_vmt_weight', 'census tract', 'hh_cnt', 'tot_pop', 'avg_weekday_person_miles',
       'avg_weekday_person_trips', 'avg_weekday_vehicle_trips',
       'tract_weekday_vmt','state','hh_w_veh','pct_hh_w_veh']]

#### Calculate fraction of LDV VMT (excluding light commercial trucks) that is personal travel. 

In [10]:
d_service_demand = pd.read_csv('../Evolved/evolved_demand_outputs/d_service_demand.csv')
d_service_demand = d_service_demand[d_service_demand['SECTOR']=='transportation']
state_transport_demand = d_service_demand[d_service_demand['WESTERN STATES']== state]

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [11]:
onroad_subsectors = ['medium duty trucks', 'light duty autos', 'light duty trucks',
       'transit buses', 'heavy duty trucks','school and intercity buses']

state_transport_demand = state_transport_demand[state_transport_demand['SUBSECTOR'].isin(onroad_subsectors)]
nan_inds = state_transport_demand['DEMAND_TECHNOLOGY'].isna()
bus_type = state_transport_demand[nan_inds]['OTHER_INDEX_1']
state_transport_demand.loc[nan_inds,'DEMAND_TECHNOLOGY'] = bus_type + ' buses'
state_transport_demand.loc[state_transport_demand['DEMAND_TECHNOLOGY']=='school buses', 'SUBSECTOR'] = state_transport_demand['DEMAND_TECHNOLOGY']
state_transport_demand.loc[state_transport_demand['DEMAND_TECHNOLOGY']=='intercity buses', 'SUBSECTOR'] = state_transport_demand['DEMAND_TECHNOLOGY']
state_transport_demand.loc[state_transport_demand['FINAL_ENERGY'].isna(), 'FINAL_ENERGY'] = 'bus_fuelmix'
mile_inds = state_transport_demand['UNIT']=='MILE'
state_transport_demand.loc[mile_inds,'VALUE']= state_transport_demand.loc[mile_inds,'VALUE']/ 1000000000
state_transport_demand.loc[mile_inds,'UNIT']= 'GIGAMILE'

In [12]:
invalid_fuel_groups = state_transport_demand.groupby(['DEMAND_TECHNOLOGY', 'FINAL_ENERGY'])['VALUE'].sum() == 0
invalid_fuel_inds = []
for i in range(state_transport_demand.shape[0]):
    row = state_transport_demand.iloc[i]
    dt = row['DEMAND_TECHNOLOGY']
    fe = row['FINAL_ENERGY']
    drop = invalid_fuel_groups[dt,fe]
    invalid_fuel_inds.append(drop)
invalid_fuel_inds = np.array(invalid_fuel_inds)
state_transport_demand = state_transport_demand[~invalid_fuel_inds]

In [13]:
lda_inds = state_transport_demand['SUBSECTOR']=='light duty autos'
ldt_inds = state_transport_demand['SUBSECTOR']=='light duty trucks'

state_transport_demand = state_transport_demand[lda_inds | ldt_inds]

In [14]:
ldv_vmt = state_transport_demand.groupby(['SCENARIO','YEAR','UNIT'])['VALUE'].sum().reset_index()
ldv_vmt['VALUE_personal'] = ldv_vmt['VALUE']* personal_travel_fraction
comm_truck_inds = state_transport_demand['OTHER_INDEX_1'] == 'light commercial trucks'
state_transport_demand = state_transport_demand[~comm_truck_inds]
ldv_vmt_nocomm = state_transport_demand.groupby(['SCENARIO','YEAR','UNIT'])['VALUE'].sum().reset_index()
ldv_vmt_nocomm['VALUE_personal'] = ldv_vmt['VALUE_personal']
ldv_vmt_nocomm['personal_fraction'] = ldv_vmt_nocomm['VALUE_personal']/ldv_vmt_nocomm['VALUE']

### Allocate ldv personal VMT to households

In [15]:
state_personal_vmt = ldv_vmt_nocomm

r = np.arange(len(state_personal_vmt)).repeat(len(state_fuel_burden))
tract_vmt_weight = pd.concat([state_fuel_burden]*len(state_personal_vmt),ignore_index=True)
tract_vmt = pd.DataFrame(state_personal_vmt.values[r], columns = state_personal_vmt.columns).join(tract_vmt_weight)

In [16]:
tract_vmt['VALUE_personal_tract'] = tract_vmt['VALUE_personal'] * tract_vmt['tract_vmt_weight'] * 1000000000
tract_vmt.rename(columns={'VALUE_personal_tract':'Personal Travel (Miles/Tract)'}, inplace=True)

In [17]:
tract_vmt = tract_vmt[['state','census tract','SCENARIO',
                    'YEAR','Personal Travel (Miles/Tract)']]

tract_vmt.columns = tract_vmt.columns.str.title()

In [18]:
tract_vmt

,State,Census Tract,Scenario,Year,Personal Travel (Miles/Tract)
0,NM,35001000108,100 PERCENT RENEWABLES,2017,2.05038e+07
1,NM,35001000109,100 PERCENT RENEWABLES,2017,1.85771e+07
2,NM,35001000110,100 PERCENT RENEWABLES,2017,2.41875e+07
3,NM,35001000111,100 PERCENT RENEWABLES,2017,2.15152e+07
4,NM,35001000113,100 PERCENT RENEWABLES,2017,2.50123e+07
...,...,...,...,...,...
84655,NM,35061970902,SLOW ELECTRICITY,2050,1.93071e+07
84656,NM,35061971000,SLOW ELECTRICITY,2050,4.26645e+07
84657,NM,35061971100,SLOW ELECTRICITY,2050,1.24132e+07
84658,NM,35061971300,SLOW ELECTRICITY,2050,1.45718e+07


In [22]:
tract_vmt.to_csv('../Transportation/my outputs/{}_transportation_tract_personalVMT.csv'.format(state))

In [19]:
d_energy = pd.read_csv('../Evolved/evolved_demand_outputs/d_energy.csv')
d_energy = d_energy[d_energy['SECTOR']=='transportation']
state_transport_energy = d_energy[d_energy['WESTERN STATES']== state]

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (4,7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [20]:
onroad_subsectors = ['medium duty trucks', 'light duty autos', 'light duty trucks',
       'transit buses', 'heavy duty trucks','school and intercity buses']

state_transport_energy = state_transport_energy[state_transport_energy['SUBSECTOR'].isin(onroad_subsectors)]
nan_inds = state_transport_energy['DEMAND_TECHNOLOGY'].isna()
bus_type = state_transport_energy[nan_inds]['OTHER_INDEX_1']
state_transport_energy.loc[nan_inds,'DEMAND_TECHNOLOGY'] = bus_type + ' buses'
state_transport_energy.loc[state_transport_energy['DEMAND_TECHNOLOGY']=='school buses', 'SUBSECTOR'] = state_transport_energy['DEMAND_TECHNOLOGY']
state_transport_energy.loc[state_transport_energy['DEMAND_TECHNOLOGY']=='intercity buses', 'SUBSECTOR'] = state_transport_energy['DEMAND_TECHNOLOGY']

#state_transport_energy.loc[state_transport_energy['FINAL_ENERGY'].isna(), 'FINAL_ENERGY'] = 'bus_fuelmix'
# mile_inds = state_transport_energy['UNIT']=='MILE'
# state_transport_energy.loc[mile_inds,'VALUE']= state_transport_energy.loc[mile_inds,'VALUE']/ 1000000000
# state_transport_energy.loc[mile_inds,'UNIT']= 'GIGAMILE'

In [21]:
invalid_fuel_groups = state_transport_energy.groupby(['DEMAND_TECHNOLOGY', 'FINAL_ENERGY'])['VALUE'].sum() == 0
invalid_fuel_inds = []
for i in range(state_transport_energy.shape[0]):
    row = state_transport_energy.iloc[i]
    dt = row['DEMAND_TECHNOLOGY']
    fe = row['FINAL_ENERGY']
    drop = invalid_fuel_groups[dt,fe]
    invalid_fuel_inds.append(drop)
invalid_fuel_inds = np.array(invalid_fuel_inds)
state_transport_energy = state_transport_energy[~invalid_fuel_inds]

In [22]:
lda_inds = state_transport_energy['SUBSECTOR']=='light duty autos'
ldt_inds = state_transport_energy['SUBSECTOR']=='light duty trucks'
state_transport_energy = state_transport_energy[lda_inds | ldt_inds]

comm_inds = state_transport_energy['OTHER_INDEX_1']=='light commercial trucks'
state_transport_energy = state_transport_energy[~comm_inds]

In [23]:
for year in state_transport_energy['YEAR'].unique():
    vmt_year_inds = ldv_vmt_nocomm['YEAR'] == year
    for scen in state_transport_energy['SCENARIO'].unique():
        vmt_scen_inds = ldv_vmt_nocomm['SCENARIO'] == scen
        #vmt_year_scen = ldv_vmt_nocomm[vmt_year_inds & vmt_scen_inds]
        personal_fraction = ldv_vmt_nocomm.loc[vmt_year_inds & vmt_scen_inds, 'personal_fraction'].item()

        energy_year_inds = state_transport_energy['YEAR'] == year
        energy_scen_inds = state_transport_energy['SCENARIO'] == scen
        
        state_transport_energy.loc[energy_year_inds & energy_scen_inds, 'VALUE_personal'] = state_transport_energy.loc[energy_year_inds & energy_scen_inds, 'VALUE'] * personal_fraction
        

In [24]:
state_transport_energy

,TIMESTAMP,SCENARIO,SECTOR,WESTERN STATES,DEMAND_TECHNOLOGY,SUBSECTOR,YEAR,OTHER_INDEX_2,OTHER_INDEX_1,FINAL_ENERGY,UNIT,VALUE,VALUE_personal
1032546,2020-12-13 23:39:00,REFERENCE,transportation,new mexico,CNG Light-Duty Auto,light duty autos,2017,NaN,NaN,compressed pipeline gas,mmBtu,6764.113814,5117.851899
1032547,2020-12-13 23:39:00,REFERENCE,transportation,new mexico,CNG Light-Duty Auto,light duty autos,2018,NaN,NaN,compressed pipeline gas,mmBtu,6438.606938,4872.881447
1032548,2020-12-13 23:39:00,REFERENCE,transportation,new mexico,CNG Light-Duty Auto,light duty autos,2019,NaN,NaN,compressed pipeline gas,mmBtu,6108.570260,4624.334958
1032549,2020-12-13 23:39:00,REFERENCE,transportation,new mexico,CNG Light-Duty Auto,light duty autos,2020,NaN,NaN,compressed pipeline gas,mmBtu,5777.870025,4375.139792
1032550,2020-12-13 23:39:00,REFERENCE,transportation,new mexico,CNG Light-Duty Auto,light duty autos,2021,NaN,NaN,compressed pipeline gas,mmBtu,5411.904887,4099.284130
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5483102,2020-12-13 23:39:00,LOW DEMAND,transportation,new mexico,Reference TDI Light-Duty Truck,light duty trucks,2046,NaN,light trucks,diesel fuel,mmBtu,107999.062010,82586.980934
5483112,2020-12-13 23:39:00,LOW DEMAND,transportation,new mexico,Reference TDI Light-Duty Truck,light duty trucks,2047,NaN,light trucks,diesel fuel,mmBtu,83901.721382,64188.030342
5483122,2020-12-13 23:39:00,LOW DEMAND,transportation,new mexico,Reference TDI Light-Duty Truck,light duty trucks,2048,NaN,light trucks,diesel fuel,mmBtu,64343.990688,49247.387416
5483132,2020-12-13 23:39:00,LOW DEMAND,transportation,new mexico,Reference TDI Light-Duty Truck,light duty trucks,2049,NaN,light trucks,diesel fuel,mmBtu,48665.566446,37263.999595


In [25]:
personal_transport_energy = state_transport_energy
personal_transport_energy.drop(columns='VALUE',inplace=True)

In [26]:
personal_transport_energy

,TIMESTAMP,SCENARIO,SECTOR,WESTERN STATES,DEMAND_TECHNOLOGY,SUBSECTOR,YEAR,OTHER_INDEX_2,OTHER_INDEX_1,FINAL_ENERGY,UNIT,VALUE_personal
1032546,2020-12-13 23:39:00,REFERENCE,transportation,new mexico,CNG Light-Duty Auto,light duty autos,2017,NaN,NaN,compressed pipeline gas,mmBtu,5117.851899
1032547,2020-12-13 23:39:00,REFERENCE,transportation,new mexico,CNG Light-Duty Auto,light duty autos,2018,NaN,NaN,compressed pipeline gas,mmBtu,4872.881447
1032548,2020-12-13 23:39:00,REFERENCE,transportation,new mexico,CNG Light-Duty Auto,light duty autos,2019,NaN,NaN,compressed pipeline gas,mmBtu,4624.334958
1032549,2020-12-13 23:39:00,REFERENCE,transportation,new mexico,CNG Light-Duty Auto,light duty autos,2020,NaN,NaN,compressed pipeline gas,mmBtu,4375.139792
1032550,2020-12-13 23:39:00,REFERENCE,transportation,new mexico,CNG Light-Duty Auto,light duty autos,2021,NaN,NaN,compressed pipeline gas,mmBtu,4099.284130
...,...,...,...,...,...,...,...,...,...,...,...,...
5483102,2020-12-13 23:39:00,LOW DEMAND,transportation,new mexico,Reference TDI Light-Duty Truck,light duty trucks,2046,NaN,light trucks,diesel fuel,mmBtu,82586.980934
5483112,2020-12-13 23:39:00,LOW DEMAND,transportation,new mexico,Reference TDI Light-Duty Truck,light duty trucks,2047,NaN,light trucks,diesel fuel,mmBtu,64188.030342
5483122,2020-12-13 23:39:00,LOW DEMAND,transportation,new mexico,Reference TDI Light-Duty Truck,light duty trucks,2048,NaN,light trucks,diesel fuel,mmBtu,49247.387416
5483132,2020-12-13 23:39:00,LOW DEMAND,transportation,new mexico,Reference TDI Light-Duty Truck,light duty trucks,2049,NaN,light trucks,diesel fuel,mmBtu,37263.999595


In [31]:
r = np.arange(len(personal_transport_energy)).repeat(len(state_fuel_burden))
fuel_burden_df = pd.concat([state_fuel_burden]*len(personal_transport_energy),ignore_index=True)
tract_fuel_burden = pd.DataFrame(personal_transport_energy.values[r], columns = personal_transport_energy.columns).join(fuel_burden_df)

In [32]:
tract_fuel_burden['VALUE_personal'] = tract_fuel_burden['VALUE_personal'] * tract_fuel_burden['tract_vmt_weight']
tract_fuel_burden.rename(columns={'VALUE_personal':'Personal fuel use (mmbtu/tract)'}, inplace=True)
#tract_fuel_burden['Fuel use (mmbtu/hh_w_veh)'] = tract_fuel_burden['Fuel use (mmbtu/tract)'] / (tract_fuel_burden['hh_cnt']- tract_fuel_burden['hh_veh_0'])

In [33]:
tract_fuel_burden = tract_fuel_burden[['state','census tract','SCENARIO', 'SECTOR',
       'DEMAND_TECHNOLOGY', 'SUBSECTOR', 'OTHER_INDEX_1','YEAR','FINAL_ENERGY', 'Personal fuel use (mmbtu/tract)', 
        'hh_w_veh','pct_hh_w_veh']]

tract_fuel_burden.columns = tract_fuel_burden.columns.str.title()
tract_fuel_burden = tract_fuel_burden.groupby(['State','Census Tract','Scenario','Sector','Year','Hh_W_Veh','Pct_Hh_W_Veh','Final_Energy'])['Personal Fuel Use (Mmbtu/Tract)'].sum().reset_index()

tract_fuel_burden_pivot = tract_fuel_burden.pivot_table(index=['State','Census Tract','Scenario','Sector','Year','Hh_W_Veh','Pct_Hh_W_Veh'], columns='Final_Energy', values='Personal Fuel Use (Mmbtu/Tract)').add_suffix(' (mmbtu/tract)').reset_index()
fuel_cols = tract_fuel_burden_pivot.columns[-5:]

tract_fuel_burden_pivot.to_csv('../Transportation/my outputs/{}_household_transportation_fueluse_pivot.csv'.format(state))

In [34]:
tract_fuel_burden_baseline = tract_fuel_burden_pivot[tract_fuel_burden_pivot['Year']==2017]
tract_fuel_burden_baseline_ref = tract_fuel_burden_baseline[tract_fuel_burden_baseline['Scenario']=='REFERENCE']
tract_fuel_burden_baseline_ref.to_csv('../Evolved/my outputs/{}_household_transportation_fueluse_baseline_ref.csv'.format(state))

### Pivot Evolved transportation fuel cost data.

In [39]:
fuel_costs = pd.read_csv('../Evolved/evolved_demand_outputs/$_per_mmbtu.csv')
fuel_costs_pivot = fuel_costs.pivot_table(index=['Western States','Sector','Scenario','Year'], columns='Final Energy', values='$/Final MMBTU').add_suffix(' ($/mmbtu)').reset_index()
fuel_costs_pivot.to_csv('../Evolved/evolved_demand_outputs/$_per_mmbtu_pivot.csv')
fuel_costs_baseline = fuel_costs_pivot[fuel_costs_pivot['Year']==2020]
fuel_costs_baseline_ref = fuel_costs_baseline[fuel_costs_baseline['Scenario']=='REFERENCE']
fuel_costs_baseline_ref.loc[fuel_costs_baseline_ref['Western States']=='colorado','Western States']='CO'
fuel_costs_baseline_ref.loc[fuel_costs_baseline_ref['Western States']=='nevada','Western States']='NV'
fuel_costs_baseline_ref.loc[fuel_costs_baseline_ref['Western States']=='new mexico','Western States']='NM'
fuel_costs_baseline_ref.to_csv('../Evolved/my outputs/fuel_costs_2020_ref.csv')

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


### Reformat Evolved residential fuel cost and Boris' residential fuel use data.

In [34]:
res_costs = pd.read_csv('../Evolved/evolved_demand_outputs/Residential fuel costs all states and years.csv')
res_costs_baseline = res_costs[res_costs['Scenario']=='BASELINE']
res_costs_baseline['Scenario'] = 'REFERENCE'
res_costs_baseline.to_csv('../Evolved/my outputs/res_fuel_costs_baseline.csv')

<ipython-input-34-a51d1919d5c9>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  res_costs_baseline['Scenario'] = 'REFERENCE'


In [35]:
res_fuel_use = pd.read_csv('../Evolved/evolved_demand_outputs/Residential tract fuel consumption all states and years.csv')
res_fuel_use_baseline = res_fuel_use[res_fuel_use['Year']==2017]
res_fuel_use_baseline['Scenario']='REFERENCE'
res_fuel_use_baseline.to_csv('../Evolved/my outputs/residential_tract_fueluse_baseline.csv')

<ipython-input-35-ce0ecd989ca7>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  res_fuel_use_baseline['Scenario']='REFERENCE'


### Read in Tableau output file to assign each census tract with transportation energy burden percentile.

In [4]:
fuel_burden = pd.read_csv('../Evolved/my outputs/NM_energyBurdens_tableauOutput.csv'.format(state_abbrev))
nan_inds = fuel_burden['Household Fuel Burden (% MHI)'].isna()
fuel_burden = fuel_burden[~nan_inds]
fuel_burden['Transportation Fuel Burden Percentile'] = fuel_burden['Household Transportation Fuel Burden (% of MHI)'].rank(pct=True)
fuel_burden['Combined Fuel Burden Percentile'] = fuel_burden['Household Fuel Burden (% MHI)'].rank(pct=True)

In [5]:
fuel_burden.to_csv('../Evolved/my outputs/{}_FuelBurden_wPercentile.csv'.format(state))

### Estimate fuel burden from weekday travel (not using Evolved VMT).

In [10]:
latch_western['Fuel Economy'] = 26.2
latch_western['Fuel Type'] = 'Gasoline'
latch_western['Source Type'] = 'Light duty auto'
latch_western['Fuel Use (gallons/weekday/hh_w_veh)'] = latch_western['avg_weekday_hh_vmt'] / latch_western['Fuel Economy']
latch_western['Fuel price (per gallon gas)'] = 2.3
latch_western['Household fuel expenditure (per weekday)'] = latch_western['Fuel price (per gallon gas)'] * latch_western['Fuel Use (gallons/weekday/hh_w_veh)']
latch_western['Annual fuel burden (weekday travel)'] = latch_western['Household fuel expenditure (per weekday)'] * 260 / latch_western['median_hh_inc']



In [12]:
latch_western['Fuel burden percentile'] = latch_western.groupby(['state'])['Annual fuel burden (weekday travel)'].rank(pct=True)

In [63]:
latch_western.to_csv('../Transportation/my outputs/latch2017_fuelburden_tract.csv')